In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
import os, json, re
from tqdm import tqdm
from dotenv import load_dotenv

load_dotenv()
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

DATA_DIR = "../data/outputs"

tag_prompt = ChatPromptTemplate.from_template("""
당신은 대한민국 복지정책 분석 전문가입니다.
다음 복지 프로그램 정보를 보고 아래 JSON 구조의 태그를 생성하세요.

입력:
{data}

출력 형식 (이 형식을 반드시 지켜주세요):
{{
  "age_group": "연령 기준. 원본 데이터에 명시된 정확한 연령 범위나 기준을 그대로 추출. 예: '만 18세 이하', '만 19세~34세', '65세 이상', '만 6세 미만', '임신부터 출산 후 12개월까지' 등. 청소년, 청년 등의 용어가 나와도 구체적 연령 범위가 함께 제시된 경우 그 수치를 우선 기재. 연령 제한이 없거나 모든 연령이 대상인 경우 '무관'으로 표시.",
  "income_level": "소득 수준 기준. 공식 소득지표 또는 상대적 소득 구간 표현. '기준중위소득 ○% 이하', '소득하위 ○%', '저소득층', '차상위계층', '기초생활수급자' 등 포함. 절대 금액(예: 200만 원 이하)도 이 항목에 포함. 소득 제한이 없는 경우 '무관'으로 표시.",
  "household_type": "가구 구성 형태. 동거 가족의 구조를 의미함. '1인가구', '부부가구', '다자녀가구', '조손가구', '한부모가구', '다문화가구' 등으로 구분. 주거지 형태(예: 전세, 월세)는 포함하지 않음. 가구 형태 제한이 없는 경우 '무관'으로 표시.",
  "region": "적용 지역 범위. 전국 단위인지, 특정 지역(광역/기초자치단체, 읍면동 등)인지 구분. '전국', '서울특별시', '경기도', '농어촌', '도서지역' 등. 행정구역 명칭만 포함하며, 기관명이나 지역기관명은 제외.",
  "target_group": "복지 정책의 주요 수혜 대상 집단. 생애주기, 사회적 지위, 직업군 등 사람의 속성으로 정의됨. '노인', '장애인', '청년', '청소년', '임산부', '영유아', '한부모', '저소득층', '국가유공자', '농어민', '구직자' 등. 단순 연령대가 아닌 사회적 분류 개념 중심.",
  "benefit_type": "복지 제도의 제공 형태 또는 지원 방식. 실제 혜택의 유형을 의미함. '현금지원', '현물지원', '감면(요금, 세금, 보험료 등)', '서비스제공(돌봄, 교육, 간병 등)', '대출지원', '의료비지원', '취업지원', '문화활동지원' 등. 단순 제도명(예: 연금제도)은 포함하지 않음.",
  "application_method": "신청 경로 또는 절차. 이용자가 직접 접속하거나 방문하는 접점 기준. '읍면동 행정복지센터', '복지로(온라인)', '국민연금공단 지사', '보건소 방문', '자동지급(별도신청 불필요)' 등. 단순히 기관명이 아니라 '신청 절차상 위치' 중심으로 구분.",
  "institution": "주관 또는 집행 기관. 복지사업의 관리 주체. '보건복지부', '고용노동부', '국민연금공단', '국민건강보험공단', '한국장애인고용공단', '지자체(서울특별시, 시군구청 등)' 등. 문의처에 공단, 공사, 센터 등의 기관 정보가 있으면 해당 기관명을 우선 기재. 전화번호만 있고 기관명이 없으면 '미상'으로 표시."
}}

규칙:
- 해당 기준이 없이 누구나 받을 수 있다면 "무관"으로 작성하세요.
- 텍스트에서 직접 근거가 없는 값은 "미상"으로 작성하세요.
- 연령, 소득, 가구형태에 제한이 명시되지 않았다면 "무관"으로 판단하세요.
- institution은 문의처에서 "공단", "공사", "센터", "부", "청" 등이 포함된 기관명을 우선 추출하세요.
- benefit_type은 구체적 금액보다는 지원 방식(현금지원, 의료비지원 등)으로 통일하세요.
- 복수 가능 시 쉼표(,)로 구분합니다.
- 가능한 한 간결하고 일관된 용어를 사용하세요.
- JSON 형식을 정확히 지켜주세요. 마지막 항목 뒤에 쉼표를 붙이지 마세요.
""")

def fix_json_response(content):
    """LLM 응답에서 trailing comma 제거"""
    content = content.strip()
    content = re.sub(r',\s*}', '}', content)
    content = re.sub(r',\s*]', ']', content)
    return content

def auto_tag_policy(item):
    """태깅 함수"""
    text_data = json.dumps({
        "title": item.get("title"),
        "target": item.get("target"),
        "content": item.get("content"),
        "method": item.get("method"),
        "inquiry": item.get("inquiry")
    }, ensure_ascii=False)
    
    messages = tag_prompt.format_messages(data=text_data)
    try:
        response = llm.invoke(messages)
        fixed_content = fix_json_response(response.content)
        tags = json.loads(fixed_content)
        item["tags"] = tags
    except Exception as e:
        print(f"태깅 실패 ({item.get('title','무제')}): {e}")
        item["tags"] = {}
    return item

target_files = [
    "생계지원_2025.json",
    "취업지원_2025.json",
    "임신보육지원_2025.json",
    "청소년청년지원_2025.json",
    "보건의료지원_2025.json",
    "노령자지원_2025.json",
    "장애인지원_2025.json"
]

for file in target_files:
    path = os.path.join(DATA_DIR, file)
    if not os.path.exists(path):
        print(f"파일 없음: {file}")
        continue

    with open(path, "r", encoding="utf-8") as f:
        data = json.load(f)

    print(f"{file} ({len(data)}건) 태깅 시작")
    tagged_data = [auto_tag_policy(item) for item in tqdm(data)]

    save_path = path.replace(".json", "_tagged.json")
    with open(save_path, "w", encoding="utf-8") as f:
        json.dump(tagged_data, f, ensure_ascii=False, indent=2)

    print(f"저장 완료: {save_path}")

생계지원_2025.json (34건) 태깅 시작


100%|██████████| 34/34 [01:41<00:00,  2.97s/it]


저장 완료: ../data/outputs/생계지원_2025_tagged.json
취업지원_2025.json (17건) 태깅 시작


100%|██████████| 17/17 [00:41<00:00,  2.43s/it]


저장 완료: ../data/outputs/취업지원_2025_tagged.json
임신보육지원_2025.json (40건) 태깅 시작


...

저장 완료: ../data/outputs/보건의료지원_2025_tagged.json
노령자지원_2025.json (19건) 태깅 시작


100%|██████████| 19/19 [00:44<00:00,  2.36s/it]


저장 완료: ../data/outputs/노령자지원_2025_tagged.json
장애인지원_2025.json (55건) 태깅 시작


100%|██████████| 55/55 [02:00<00:00,  2.18s/it]

저장 완료: ../data/outputs/장애인지원_2025_tagged.json
